# SubmodularGreedy.jl Tutorial

This Jupyter notebook contains a tutorial for the `SubmodularGreedy.jl` package, available [here](https://github.com/crharshaw/SubmodularGreedy.jl).
In this tutorial, you will learn how to 

1. Create function and independence oracles required for `SubmodularGreedy.jl`
2. Call basic versions of the Greedy, RepeatedGreedy, SimultaneousGreedy, and SampleGreedy algorithms.
3. Call the variants that run in linear time and handle knapsack constriaints.
4. Apply the package to a problem with real data: Movie Recommendation

## Table of Contents

1. A brief introduction to submodular optimization
2. Installing and loading the package
3. Running the algorithms on the simple instance
4. Automatic parameter tuning
5. Linear time implementation
6. Knapsack constraints
7. Movie Recommendation: a problem instance with real data

## A brief introduction to submodular optimization

**Note** If you are familiar with submodular optimization, you can skip this section; otherwise, please read on.

We begin by describing the mathematical formalization of submodular optimization. 
Let $\mathcal{N}$ be a finite set of size $n$ known as the *ground set*.
Without loss of generality, we may consider the ground set to be integers $\mathcal{N} = \{ 1, 2, \dots n\}$.
We consider a function $f: 2^{\mathcal{N}} \rightarrow \mathbb{R}$ defined on subsets of the ground set, which will serve as our objective.
A function $f$ is submodular if it satisfies the diminishing returns property,

$$
f(A \cup \{e\}) - f(A) \geq f(B \cup \{e\}) - f(B) \quad \text{ for all } A \subset B \subset \mathcal{N}, e \notin B.
$$

We write the marignal gain of adding element $e$ to set $A$ as $f(e \mid A) = f(A \cup \{e\}) - f(A)$.

Our constraints are given by an *independence system* denoted $(\mathcal{N}, \mathcal{I})$, where $\mathcal{I} \subset \mathcal{N}$.
The pair $(\mathcal{N}, \mathcal{I})$ forms an independence system if it satisfies the down-closed property that 

$$
A \in \mathcal{I} \text{ and } B \subset A \text{ implies that } B \in \mathcal{I}.
$$

The submodular maximization problem is to find a feasible set $S \in \mathcal{I}$ which maximizes the objective $f(S)$; formally, 

$$
\max_{S \in \mathcal{I}} f(S) \enspace.
$$

The `SubmodularGreedy.jl` package contains fast implementations of greedy-based algorithms to solve the problem above.

A priori, the objective function and the constraint set could have an exponentially large description size, which would preclude any reasonably fast algorithm for optimization.
For this reason, the vast majority of the literature on submodular optimization assumes *oracle access* to the objective function and the independence system.
That is, it is assumed that an algorithm for submodular optimization has access to two subroutines:

1. **Value Oracle**: Given $S \subset \mathcal{N}$, return the value $f(S)$
2. **Independence Oracle** Given $S \subset \mathcal{N}$ return `true` if $S \in \mathcal{I}$ and `false` otherwise.

All algorithms in this package are greedy-based algorithms and so we will actually only need the following two oracles, which are usually cheaper to implement:

1. **Marginal Gain Oracle**: Given $S \subset \mathcal{N}$ and $e \in \mathcal{N}$, return the value of the marginal gain $f(e \mid A)$.
2. **Marginal Independence Oracle**: Given $S \in \mathcal{I}$ and $e \in \mathcal{N}$, return `true` if $S \cup \{e\} \in \mathcal{I}$ and `false` otherwise.

When using the `SubmodularGreedy.jl` package, the user will provide the two oracles above.
Faster implementation of these oracles will yield to faster runtimes of the greedy algorithms provided in this package.
In this next section of code, we will see examples of how to code these oracles.

## Installing and loading the package

To begin, you should have installed the `SubmodularGreedy.jl` package.
See the directions on the GitHub page [here](https://github.com/crharshaw/SubmodularGreedy.jl) to see how to do this.

Once you have installed the package, you can load it using the following command.

In [1]:
using SubmodularGreedy

We also include several other packages which will be helpful when we get to the 

In [2]:
using Random
using JLD2
using DataFrames
using LinearAlgebra

## Running the algorithms on the simple instance

We begin the tutorial by demonstrating how to run the greedy optimization algorithms in `SubmodularGreedy.jl` on the following simple problem instance:

$$
\max_{|S| \leq k} f(S) \triangleq \sum_{e \in S} c_e + \frac{1}{n} \sqrt{|S|} \enspace,
$$

where the coefficients are given by $c_e = e$, i.e. $c_1 = 1$, $c_2 = 2$, etc.
This function is the sum of a modular function and a concave function, where the concave part contributes only a very small fraction of the overall value.
In this sense, the function is essentially modular.

We are using this function in the tutorial because it is simple enough that one can reason about the execution path of the various algorithms in the package.
In particular, the element with largest marginal gain will always be the element with the largest index; in the case of Simultaneous Greedy, the element-solution pair with largest marginal gain will be the element with largest index and the solution with fewest elements.

### Creating function and independence oracles

The ground set $\mathcal{N}$ is always specified as an array of integers.

A priori, the objective function $f(S)$ and the independence system $(\mathcal{N}, \mathcal{I})$ could have an exponentially large description size, which would preclude any reasonably fast algorithm for optimization.
For this reason, the vast majority of the literature on submodular optimization assumes *oracle access* to the objective function and the independence system.
That is, it is assumed that an algorithm for submodular optimization has access to two subroutines:

1. **Value Oracle**: Given $S \subset \mathcal{N}$, return the value $f(S)$
2. **Independence Oracle** Given $S \subset \mathcal{N}$ return `true` if $S \in \mathcal{I}$ and `false` otherwise.

All algorithms in this package are greedy-based algorithms and so we will actually only need the following two oracles, which are usually cheaper to implement:

1. **Marginal Gain Oracle**: Given $S \subset \mathcal{N}$ and $e \in \mathcal{N}$, return the value of the marginal gain $f(e \mid A)$.
2. **Marginal Independence Oracle**: Given $S \in \mathcal{I}$ and $e \in \mathcal{N}$, return `true` if $S \cup \{e\} \in \mathcal{I}$ and `false` otherwise.

When using the `SubmodularGreedy.jl` package, the user will provide the two oracles above.
Faster implementation of these oracles will yield to faster runtimes of the greedy algorithms provided in this package.
In this next section of code, we will see examples of how to code these oracles.

### A simple objective: linear plus concave

Let's begin by implementing a value oracle for the objective function,

$$
f(S) = \sum_{e \in S} c_e + \frac{1}{n} \cdot \sqrt{|S|} \enspace.
$$

In [3]:
function simple_obj(S::Set{Int64}, n::Int64)
    return sum(e for e in S) + sqrt(length(S)) / n
end

simple_obj (generic function with 1 method)

Let's verify it returns the right values for $S = \{1,2,3\}$ and $S = \{5\}$ when the ground set is $n=10$.

In [4]:
n = 10
S = Set([1,2,3])
simple_obj(S, n)

6.173205080756888

In [5]:
n = 10
S = Set([5])
simple_obj(S, n)

5.1

As discussed above, a marginal gain oracle can often be implemented to be faster than a value oracle.
In particular, the value oracle `f` above requires $\mathcal{O}(|S|)$ additions.
On the other hand, the following marginal gain oracle requires only a constant number of additions.

In [6]:
function simple_obj_diff(e::Int64, S::Set{Int64}, n::Int64)
   if e in S
        return 0
    else
        k = length(S)
        return e + (sqrt(k+1) - sqrt(k)) / n
    end
end

simple_obj_diff (generic function with 1 method)

Let's quickly verify its correctness by using the value oracle.

In [7]:
n = 10
e = 6
S = Set([1,2,3])

gain1 = simple_obj(union(S,e),n) - f(S, n)
gain2 = simple_obj_diff(e, S, n)

println("Gain 1 is $gain1 and Gain 2 is $gain2")

LoadError: UndefVarError: f not defined

The greedy optimization algorithms in the `SubmodularGreedy.jl` package only require marginal gain oracles, and so we shall use these throughout the remainder of the tutorial.
Again, we stress that although one may always obtain a marginal gain oracle from two calls to a value oracle, this is rarely the most efficient implementation in practice.

### A simple independence system: cardinality constraint

We continue our example with a simple independence system: the cardinality constraint.
Below is an implementation of the indpendence oracle.

In [8]:
function card_oracle(S::Set{Int64}, k::Int64)
    return length(S) <= k
end

card_oracle (generic function with 1 method)

In [9]:
S = Set([1,2,3])
k = 10

card_oracle(S,k)

true

In [10]:
S = Set([1,2,3,5,6])
k = 2

card_oracle(S,k)

false

The greedy optimization algorithms in the `SubmodularGreedy.jl` package require a marginal independence oracle.
In this case, the implementation of the marginal independence oracle will look very similar to the independence oracle.

In [11]:
function marginal_card_oracle(e::Int64, S::Set{Int64}, k::Int64)
    return length(union(S,e)) <= k
end

marginal_card_oracle (generic function with 1 method)

In [12]:
e = 4
S = Set([1,2,3])
k = 4

marginal_card_oracle(e,S,k)

true

In [13]:
e = 4
S = Set([1,2,3])
k = 3

marginal_card_oracle(e,S,k)

false

Using the marginal gain oracles and the marginal independence oracles, we will run the Simultaneous Greedy, Repeated Greedy, and Greedy algorithms on our simple problem instance:

$$
\max_{|S| \leq k} \sum_{e \in S} c_e + \frac{1}{n} \sqrt{|S|} \enspace,
$$

where the coefficients are given by $c_e = e$, i.e. $c_1 = 1$, $c_2 = 2$, \dots $c_n = n$, etc.
We fix a ground set of size $n = 10$, a cardinality constraint of $k=3$, and we instantiate the necessary oracles below:

In [14]:
n = 10
k = 3

gnd = collect(1:n)
f_diff(e,S) = simple_obj_diff(e,S,n)
ind_add_oracle(e,S) = marginal_card_oracle(e,S,k)

ind_add_oracle (generic function with 1 method)

### Greedy

Let's begin by running the greedy algorithm.
The greedy algorithm should construct the solution

$$
S = \{10, 9, 8\} \enspace.
$$

To run the greedy algorithm, we call the function `greedy` with inputs:

1. `gnd`, an array of ground set elements
2. `f_diff`, a marginal gain oracle
3. `ind_add_oracle`, a marginal independence oracle

In [15]:
sol, f_val, num_fun, num_oracle, knap_reject = greedy(gnd, f_diff, ind_add_oracle, verbose=true)


Iteration 1
	Set 1 is { } with value 0.0

	Looking at element 10 with solution 1, previously queried at 0 with gain 10.1 and density 0.0
		We will be adding this element 10 to solution 1

Iteration 2
	Set 1 is { 10 } with value 10.1

	Looking at element 9 with solution 1, previously queried at 0 with gain 9.1 and density 0.0
	Looking at element 9 with solution 1, previously queried at 1 with gain 9.04142135623731 and density 0.0
		We will be adding this element 9 to solution 1

Iteration 3
	Set 1 is { 9, 10 } with value 19.14142135623731

	Looking at element 8 with solution 1, previously queried at 0 with gain 8.1 and density 0.0
	Looking at element 8 with solution 1, previously queried at 2 with gain 8.031783724519578 and density 0.0
		We will be adding this element 8 to solution 1

Iteration 4
	Set 1 is { 8, 9, 10 } with value 27.173205080756887

	Looking at element 7 with solution 1, previously queried at 0 with gain 7.1 and density 0.0
	Looking at element 6 with solution 1, previo

(Set([9, 10, 8]), 27.173205080756887, 12, 19, false)

We see from the printed execution path that Greedy runs as expected.
When we executed the function above, we set `verbose=true`, which displays the entire execution path.
The default is `verobse=false`, which does not print anything.

Another thing we see from the execution path is that few marginal gain oracles are needed in this problem instance.
In particular, the marginal gain oracle is called only $n + (k-1) = 10 + (3-1) = 12$ times, once to create the priority queue and once at each iteration after the first.
If we used a linear search at each iteration, this number of oracle calls would be $k \cdot n = 3 \times 10 = 30$.

The output of the greedy algorithm is:

1. `sol`: the greedy solution
2. `f_val`: the objective value of the greedy solution
3. `num_fun`: the number of calls to the marginal gain oracle
4. `num_oracle`: the number of calls to the marginal independence oracle
5. `knap_reject`: (irrelevant at this point)

All other greedy optimization algorithms in the `SubmodularGreedy.jl` package have similar outputs.

If we want to see the documentation for this function, we can type `?greedy`, which brings up the following documentation:

In [16]:
?greedy

search: greedy sample_greedy repeated_greedy SubmodularGreedy



```
greedy([gnd, pq], f_diff, ind_add_oracle; knapsack_constraints=nothing, density_ratio=0.0)
```

The greedy algorithm, possibly with density ratio thresholding for knapsack constraints.

# Arguments

  * `gnd`: a integer array of elements, or a priority queue of elements
  * `f_diff`: a marginal difference oracle
  * `ind_add_oracle`: an independence oracle

# Optional Arguments

  * `knapsack_constraints`: a 2D array of knapsack constraints (default: nothing)
  * `density_ratio`: the maximum density ratio threshold (default: 0.0)
  * `epsilon`: a number in [0,1] which controls the approximation / speed trade off. Set to `0.0` for exact algorithm.
  * `opt_size_ub`: an upper bound on the size of the optimal set

# Output

  * `best_sol`: the best solution
  * `best_f_val`: value of the objective at the solution
  * `num_fun`: the number of function evaluations
  * `num_oracle`: the number of independence oracle evaluations
  * `knap_reject`: an indicator whether the knapsack feasibility was rejected or not


## Sample Greedy

Next, we run the sample greedy algorithm. 
The sample greedy algorithm takes an additional argument `sample_prob`, which is the probability that an element is included in the subsampled ground set.

We keep the verbosity level set so we can view the execution path of the algorithm.

In [17]:
Random.seed!(0)

sample_prob = 0.5
sol, f_val, num_fun, num_oracle = sample_greedy(gnd, sample_prob, f_diff, ind_add_oracle, verbose=true)

The subsampled elements are [3, 4, 5, 6, 7, 8, 9]
Iteration 1
	Set 1 is { } with value 0.0

	Looking at element 9 with solution 1, previously queried at 0 with gain 9.1 and density 0.0
		We will be adding this element 9 to solution 1

Iteration 2
	Set 1 is { 9 } with value 9.1

	Looking at element 8 with solution 1, previously queried at 0 with gain 8.1 and density 0.0
	Looking at element 8 with solution 1, previously queried at 1 with gain 8.04142135623731 and density 0.0
		We will be adding this element 8 to solution 1

Iteration 3
	Set 1 is { 8, 9 } with value 17.14142135623731

	Looking at element 7 with solution 1, previously queried at 0 with gain 7.1 and density 0.0
	Looking at element 7 with solution 1, previously queried at 2 with gain 7.0317837245195784 and density 0.0
		We will be adding this element 7 to solution 1

Iteration 4
	Set 1 is { 7, 8, 9 } with value 24.173205080756887

	Looking at element 6 with solution 1, previously queried at 0 with gain 6.1 and density 0.0
	L

(Set([7, 9, 8]), 24.173205080756887, 9, 13)

## Repeated Greedy

Next, we run the Repeated Greedy algorithm.
The additional input parameter that we have to specify is `num_sol`, the number of solutions.

In our simple instance, the Repeated Greedy algorithm will construct sets 

$$
S_1 = \{10, 9, 8\}, \quad
S_2 = \{7,6,5\}, \quad
S_3 = \{4,3,2\}
$$

and so the set $S_1 = \{10, 9, 8\}$ will be returned.
Let's set a higher verbose option so we can see the execution of the algorithm.

In [18]:
sol, f_val, num_fun, num_oracle = repeated_greedy(gnd, f_diff, ind_add_oracle, num_sol=3, verbose_lvl=2)

Running repeated greedys
Ground set has 10 elements
The independence system parameter k is not specified

Constructing 3 solutions

Iteration 1
	Greedy returned set { 8, 9, 10 } with value 27.173205080756887
	Unconstrained returned set { 8, 9, 10 } with value 27.173205080756887

Iteration 2
	Greedy returned set { 5, 6, 7 } with value 18.173205080756887
	Unconstrained returned set { 5, 6, 7 } with value 18.173205080756887

Iteration 3
	Greedy returned set { 2, 3, 4 } with value 9.173205080756887
	Unconstrained returned set { 2, 3, 4 } with value 9.173205080756887


Obtained solution S = { 8, 9, 10 }
Obtained solution has value 27.173205080756887
Required 34 function evaluations and 28 independence queries


(Set([9, 10, 8]), 27.173205080756887, 34, 28)

## Simultaneous Greedy

Finally, we run the Simultaneous Greedy algorithm.
The Simultaneous Greedy algorithm will construct different sets on this simple problem instance.
When `num_sol=3`, then we will have that

$$
S_1 = \{10, 7, 4 \} \quad
S_2 = \{9, 6, 3 \}\quad
S_3 = \{8, 5, 2 \}
$$

and the set $S_1 = \{ 10, 7, 4\}$ will be returned.
Let's set the higher verbose option so that we can see the execution of the algorithm.

In [19]:
sol, f_val, num_fun, num_oracle = simultaneous_greedys(gnd, f_diff, ind_add_oracle, num_sol=3, verbose_lvl=2)

Running simultaneous greedys
Ground set has 10 elements
Objective function is non-monotone
The independence system parameter k is not specified

Constructing 3 solutions

Iteration 1
	Set 1 is { } with value 0.0
	Set 2 is { } with value 0.0
	Set 3 is { } with value 0.0

	Looking at element 10 with solution 1, previously queried at 0 with gain 10.1 and density 0.0
		We will be adding this element 10 to solution 1

Iteration 2
	Set 1 is { 10 } with value 10.1
	Set 2 is { } with value 0.0
	Set 3 is { } with value 0.0

	Looking at element 9 with solution 1, previously queried at 0 with gain 9.1 and density 0.0
	Looking at element 9 with solution 2, previously queried at 0 with gain 9.1 and density 0.0
		We will be adding this element 9 to solution 2

Iteration 3
	Set 1 is { 10 } with value 10.1
	Set 2 is { 9 } with value 9.1
	Set 3 is { } with value 0.0

	Looking at element 8 with solution 1, previously queried at 0 with gain 8.1 and density 0.0
	Looking at element 8 with solution 3, previ

(Set([7, 4, 10]), 21.173205080756887, 27, 33)

## Automatic Parameter Tuning

In our accompanying papers, we derive settings of `num_sol` and `sample_prob`, which maximize the worst-case approximation ratio of the algorithms.
The settings here are based on the following aspects of the problem:

1. The monotonicity of the objective
2. The independence system parameter $k$
3. Whether the independence system is $k$-extedible or a $k$-system

For automatic parameter tuning, all that is required is the value $k$.
If the optional `monotone` and `extedible` arguments are not given, then they are assumed to both be false.

Let's see an example of automatic parameter tuning.
Our simple example is monotone with a $1$-extendible system.
In this case, the worst-case analysis suggests that both repeated greedy and simulatneous greedy use `num_sol=1` solution.

In [20]:
sol, f_val, num_fun, num_oracle = simultaneous_greedys(gnd, f_diff, ind_add_oracle, k=1, extendible=true, monotone=true, verbose_lvl=2)

Running simultaneous greedys
Ground set has 10 elements
Objective function is monotone
Independence system is 1-extendible system

Constructing 1 solutions

Iteration 1
	Set 1 is { } with value 0.0

	Looking at element 10 with solution 1, previously queried at 0 with gain 10.1 and density 0.0
		We will be adding this element 10 to solution 1

Iteration 2
	Set 1 is { 10 } with value 10.1

	Looking at element 9 with solution 1, previously queried at 0 with gain 9.1 and density 0.0
	Looking at element 9 with solution 1, previously queried at 1 with gain 9.04142135623731 and density 0.0
		We will be adding this element 9 to solution 1

Iteration 3
	Set 1 is { 9, 10 } with value 19.14142135623731

	Looking at element 8 with solution 1, previously queried at 0 with gain 8.1 and density 0.0
	Looking at element 8 with solution 1, previously queried at 2 with gain 8.031783724519578 and density 0.0
		We will be adding this element 8 to solution 1

Iteration 4
	Set 1 is { 8, 9, 10 } with value 27

(Set([9, 10, 8]), 27.173205080756887, 12, 19)

If, on the other hand, we had a monotone objective (which we don't) then we could set `monotone=false` or simply not specify this optional input.
In this case, the suggested number of solutions is `num_sol=2`.

In [21]:
sol, f_val, num_fun, num_oracle = simultaneous_greedys(gnd, f_diff, ind_add_oracle, k=1, extendible=true, verbose_lvl=2)

Running simultaneous greedys
Ground set has 10 elements
Objective function is non-monotone
Independence system is 1-extendible system

Constructing 2 solutions

Iteration 1
	Set 1 is { } with value 0.0
	Set 2 is { } with value 0.0

	Looking at element 10 with solution 1, previously queried at 0 with gain 10.1 and density 0.0
		We will be adding this element 10 to solution 1

Iteration 2
	Set 1 is { 10 } with value 10.1
	Set 2 is { } with value 0.0

	Looking at element 9 with solution 1, previously queried at 0 with gain 9.1 and density 0.0
	Looking at element 9 with solution 2, previously queried at 0 with gain 9.1 and density 0.0
		We will be adding this element 9 to solution 2

Iteration 3
	Set 1 is { 10 } with value 10.1
	Set 2 is { 9 } with value 9.1

	Looking at element 8 with solution 2, previously queried at 0 with gain 8.1 and density 0.0
	Looking at element 8 with solution 1, previously queried at 0 with gain 8.1 and density 0.0
	Looking at element 8 with solution 2, previousl

(Set([7, 10, 6]), 23.173205080756887, 18, 27)

Note that in both of these executions, `num_sol` was not speficied, but rather was automatically set.
The Sample Greedy and Repeated Greedy algorithms have similar automatic parameter setting features.

Although automatic parameter setting is available, we often find that tuning these parameter is desirable.
This is dicussed more in Section~XYZ of our paper.

## Linear time implementations

All of the algorithms in the `SubmodularGreedy.jl` package have the option to incorporate the marginal gain thresholding technique, introduced by [Badanidiyuru & Vondrak, SODA 2014](https://theory.stanford.edu/~jvondrak/data/submod-fast.pdf).
Roughly speaking, the technique takes as input a parameter $\epsilon \in (0,1]$ and ensures that the algorithm runs in time $\mathcal{\tilde{O}}(n / \epsilon)$, while only losing a small multiplicative factor of $(1 - \epsilon)$ in the worst-case approximation ratio.
In this sense, the input parameter $\epsilon$ trades off computation time for accuracy.

To run any algorithm with this marginal gain thresholding technique, we can simply set the optional argument `epsilon`.

In [22]:
sol, f_val, num_fun, num_oracle, knap_reject = greedy(gnd, f_diff, ind_add_oracle, epsilon=0.1, verbose=true)


Iteration 1
	Set 1 is { } with value 0.0

	Looking at element 10 with solution 1, previously queried at 0 with gain 10.1 and density 0.0
		We will be adding this element 10 to solution 1

Iteration 2
	Set 1 is { 10 } with value 10.1

	Looking at element 9 with solution 1, previously queried at 0 with gain 9.1 and density 0.0
	Looking at element 9 with solution 1, previously queried at 1 with gain 9.04142135623731 and density 0.0
		We will be adding this element 9 to solution 1
The updated threshold is 8.1 


Iteration 3
	Set 1 is { 9, 10 } with value 19.14142135623731

	Looking at element 8 with solution 1, previously queried at 0 with gain 8.1 and density 0.0
The updated threshold is 8.1 

	Looking at element 8 with solution 1, previously queried at 2 with gain 8.031783724519578 and density 0.0
		We will be adding this element 8 to solution 1
The updated threshold is 7.1 


Iteration 4
	Set 1 is { 8, 9, 10 } with value 27.173205080756887

	Looking at element 7 with solution 1, previo

(Set([9, 10, 8]), 27.173205080756887, 12, 19, false)

Note that the output printed the value of the threshold each time it was updated; however, the execution path of the greedy algorithm is the unaffected by the setting of this parameter `epsilon`.
That is because we are using the lazy greedy technique together with a nearly linear objective.

When we run the algorithms on a larger and more complex problem instance, then we will see the benefit of the linear time implementation.

## Knapsack Constraints

All of the greedy-based optimization routines in `SubmodularGreedy.jl` support knapsack constraints.
However, only Repeated Greedy and Simultaneous Greedy feature support for the density-threshold technique, introduced by [Mirzasoleiman et al, ICML 2016](http://proceedings.mlr.press/v48/mirzasoleiman16.html).
Roughly speaking, the density-threshold technique determines whether an element should be added to the current solution based not only on its marginal gain alone, but whether its marginal gain is significantly larger than its knapsack cost.
The notion of "sufficiently" here is the density threshold, which is given by $\tau$.
This threshold $\tau$ is not given as input to the algorithm, but rather, a search over possible values of $\tau$ takes place, and the best solution found is returned.
Originally, a grid search was proposed by Mirzasoleiman et al, ICML 2016; however, in our accompanying paper, we introduce a binary search which achieves the same worst-case performance and provides an exponential speed up.

The parameter for the search is `epsilon`, the same value that was used for the linear time implementation.
Setting `epsilon` smaller yields a higher accuracy solution, at the cost of a longer computation time.

### Updated Problem Instance

We will add the following knapsack constraint to our problem instance:

$$
c(S) = \sum_{e \in S} c'_e \leq 1 \enspace,
$$

where the coefficients $c'_e$ are given by 

$$
c'_e = 
\left\{
     \begin{array}{lr}
       1 & \text{ if } e \geq 8\\
       1/4 & \text{ otherwise}
     \end{array}
   \right.
$$

This knapsack constraint means if an element of $\{ 8, 9, 10\}$ appears in a solution, then that is the only element in that solution.
The greedy algorithm will get stuck with solution $S = \{10\}$, whereas the optimal solution will be $S = \{5,6,7\}$.

We encode knapsack constraints $c_j(S) \leq 1$ for $j=1,2, \dots m$ as an $m$-by-$n$ array, where the $(j,i)$ entry contains the coefficient for element $i$ in knapsack $j$.
Moreover, all of the right hand side is assumed to be 1.
For example, we give this knapsack constraint below:

In [23]:
knapsack_constraints = 0.25 * ones(2,n)
knapsack_constraints[1,10] = 0.9
knapsack_constraints[2,7:8] .= 0.7

knapsack_constraints

2×10 Array{Float64,2}:
 0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.9
 0.25  0.25  0.25  0.25  0.25  0.25  0.7   0.7   0.25  0.25

Now, let's run the greedy algorithm.

In [24]:
sol, f_val, num_fun, num_oracle, knap_reject = greedy(gnd, f_diff, ind_add_oracle, knapsack_constraints=knapsack_constraints, verbose=true)


Iteration 1
	Set 1 is { } with value 0.0

	Looking at element 10 with solution 1, previously queried at 0 with gain 10.1 and density 1.15
		We will be adding this element 10 to solution 1

Iteration 2
	Set 1 is { 10 } with value 10.1

	Looking at element 9 with solution 1, previously queried at 0 with gain 9.1 and density 0.5
	Looking at element 9 with solution 1, previously queried at 1 with gain 9.04142135623731 and density 0.5
	Looking at element 8 with solution 1, previously queried at 0 with gain 8.1 and density 0.95
	Looking at element 8 with solution 1, previously queried at 1 with gain 8.04142135623731 and density 0.95
	Looking at element 7 with solution 1, previously queried at 0 with gain 7.1 and density 0.95
	Looking at element 7 with solution 1, previously queried at 1 with gain 7.04142135623731 and density 0.95
	Looking at element 6 with solution 1, previously queried at 0 with gain 6.1 and density 0.5
	Looking at element 6 with solution 1, previously queried at 1 with ga

(Set([10]), 10.1, 19, 19, false)

As expected, the greedy algorithm gets stuck, leading to a non-optimal solution.

We can overcome this via using the density thresholding technique in either Repeated Greedy or Simultaneous Greedy.
For purposes of exposition, let's check out the Repeated Greedy algorithm.

In [25]:
repeated_greedy(gnd, f_diff, ind_add_oracle, knapsack_constraints=knapsack_constraints, k=1, num_sol=2, monotone=true, epsilon=0.5, verbose_lvl=2)

Running repeated greedys
Ground set has 10 elements
Independence system is 1-system
There are 2 knapsack constraints

Constructing 2 solutions
A grid search for the best density ratio will be run with the parameters:
	beta scaling term is 0.06666666666666667
	error term is 0.5
	bound on largest set is 10

Iteration 1 
	Upper density ratio is 6.733333333333333 and lower density ratio is 0.6733333333333333
	Best value seen is -Inf
	So far, we have 0 function evaluations and 0 independence queries

Iteration 2 
	Upper density ratio is 2.129266957846709 and lower density ratio is 0.6733333333333333
	Best value seen is 17.14142135623731
	So far, we have 23 function evaluations and 17 independence queries

Iteration 3 
	Upper density ratio is 2.129266957846709 and lower density ratio is 1.1973748027595414
	Best value seen is 17.14142135623731
	So far, we have 46 function evaluations and 34 independence queries


Obtained solution S = { 8, 9 }
Obtained solution has value 17.14142135623731
Req

(Set([9, 8]), 17.14142135623731, 79, 61)

Repeated Greedy is able to get out of this local minima and find the best solution, thanks to the higher number of solutions and the density thresholding technique.

## Movie Recommendation: a problem instance with real data

Finally, let's see how to run these algorithms on a problem instance with real data.
We are going to use Simultion 2 from the accompanying paper, so please read Section~XYZ of our paper for more information and analysis on this problem instance.

### Movie Lens + MetaData

We begin by loading the DataFrame with the data. 
Let's look at it first before describing it.

In [26]:
data_file = "movie_info.jld2"
movie_info_df = load(data_file)["movie_info_df"]

,movieId,title,genres
,Int64,String,String
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,2,Jumanji (1995),Adventure|Children|Fantasy
3,3,Grumpier Old Men (1995),Comedy|Romance
4,4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,5,Father of the Bride Part II (1995),Comedy
6,6,Heat (1995),Action|Crime|Thriller
7,7,Sabrina (1995),Comedy|Romance
8,8,Tom and Huck (1995),Adventure|Children
9,9,Sudden Death (1995),Action


In [27]:
n = size(movie_info_df,1)
names(movie_info_df)

7-element Array{String,1}:
 "movieId"
 "title"
 "genres"
 "genre_set"
 "vec"
 "year"
 "rating"

In our dataset, we have 10,473 movies, each with one of the following values:

- `movieId`: unique identifier of the movie (an integer)
- `title`: the title of the movie (string)
- `genres`: a string of genres (string, not parsed)
- `genre_list`: a list of genres (array of strings, parsed)
- `vec`: a low-dimensional representation of the movie based on user ratings (array)
- `year`: the year the movie was released (int)
- `rating`: the average IMDb rating (float)

### Constructing Objective Function

Our goal will be to use the following objective function:

$$
f(S) 
= \frac{1}{n} \left[ 
    \sum_{i \in \mathcal{N}} \sum_{j \in S} s_{i,j} 
    - \sum_{i \in S} \sum_{j \in S} s_{i,j} 
\right] \enspace,
$$

where the similarity scores $s_{i,j}$ are defined as

$$
s_{i,j} = \exp \left( - \sigma^2 (1 - \cos(v_i, v_j)) \right) \enspace,
$$

where $v_i$ is the low-dimensional representation vector of movie $i$, $\cos(v_i,v_j) = \langle v_i, v_j \rangle / (\| v_i \| \|v_j \|)$ is the cosine similarity and $\sigma > 0$ is a user-defined bandwidth parameter which controls the decay of this similarity.
We will set $\sigma = 1.0$.

Let's begin by constructing these similarities.

In [28]:
function ij_to_ind(i::Integer, j::Integer, n::Integer)

    # ensure that i < j
    if i > j
        i,j = j,i
    end

    # map (i,j) to the correct entry in the upper triangular array
    return (i-1)*n - div(i*(i-1),2) + j
end

function get_similarity(similarity_array::Array{<:AbstractFloat}, i::Integer, j::Integer, n::Integer)
    ind = ij_to_ind(i,j,n)
    return similarity_array[ind]
end

function compute_similarity_array(vec_list; sigma::AbstractFloat=1.0)

    # get dimensions
    n = length(vec_list)

    # initialize array of pair-wise similarities
    n_pairs = div(n*(n+1), 2)
    similarity_array = zeros(n_pairs)

    # fill in the pair-wise similarities
    ind = 1
    for i=1:n
        vi = vec_list[i]
        for j=i:n
            vj = vec_list[j]
            # similarity_array[ind] = exp(- lambda^2 * sum( (vi - vj).^2 ))
            similarity_array[ind] = exp(- sigma^2 * ( 1 - (dot(vi, vj) / (norm(vi) * norm(vj))) ) )
            ind += 1
        end
    end

    return similarity_array
end

compute_similarity_array (generic function with 1 method)

In [29]:
similarity_array = compute_similarity_array(Array(movie_info_df[!,:vec]))

54847101-element Array{Float64,1}:
 1.0000000000000002
 0.8942586417274286
 0.8566635893804541
 0.7860967682077294
 0.8412857663737342
 0.851589781648117
 0.8759280019736603
 0.8205142510054846
 0.7507813993489272
 0.8915876487739497
 0.9206211373091059
 0.6848410853940632
 0.8993970748546163
 ⋮
 0.8110426760199305
 0.5895017874717636
 0.9999999999999997
 0.8621615954518614
 0.7942023015157453
 0.5608411784112787
 0.9999999999999999
 0.8200394174170812
 0.6345993452545691
 0.9999999999999998
 0.6917375229176626
 1.0000000000000002

Next, let's construct the marginal gain oracle for this objective function.

In [30]:
function dispersion_diff(elm::Integer, sol::Set{<:Integer}, similarity_array::Array{<:AbstractFloat}, n::Integer)

    if elm in sol
        return 0.0
    else 

        # compute coverage and diversity terms
        coverage_term = sum([ get_similarity(similarity_array, i, elm, n) for i=1:n ])
        diversity_term = ( 2 * sum([ get_similarity(similarity_array, i, elm, n) for i in sol]) + get_similarity(similarity_array, elm, elm, n))
        return (coverage_term - diversity_term) / n
    end
end

dispersion_diff (generic function with 1 method)

In [31]:
f_diff(elm, sol) = dispersion_diff(elm, sol, similarity_array, n)

f_diff (generic function with 1 method)

### Constructing Constraints

Next, we will consider the following $2$-extendible system: a solution $S$ is independent 

To construct constraints, we use release date constraint (independence system) and a rating budget (knapsack constraint), defined as follows.
For each movie $e \in \mathcal{N}$, let $y_e$ denote the release year of the movie.
We define the \emph{release date constraint} $(\mathcal{N}, \mathcal{I})$, where $S \in \mathcal{I}$ if

$$
|y_e - y_u| \geq 1 \text{ for all pairs } e,u \in S \enspace. 
$$

In other words, no two movies in a feasible solution set may be released in the same year.
This independence set is $2$-extendible, as adding a movie $e$ to the current solution requires the removal of up to $2$ other movies  that already belong to the set: one that appears up to a year after $y_e$ and one that appears up to a year before $y_e$.

Below is an independence oracle for the release date constraint:

In [32]:
function years_apart_feasible(sol::Set{<:Integer}, cardinality_limit::Integer, years_apart::Integer, date_list::Array{<:Integer})

    if length(sol) > cardinality_limit
        return false 
    end

    if years_apart > 0
        sol_dates = sort([date_list[elm] for elm in sol])
        for i=1:(length(sol)-1)
            if sol_dates[i+1] - sol_dates[i] < years_apart
                return false
            end
        end
    end

    return true
end

years_apart_feasible (generic function with 1 method)

In [33]:
date_list = Array(movie_info_df[!, :year])

card_limit = 100
years_apart = 1

ind_add_oracle(elm, sol) = years_apart_feasible(union(sol, elm), card_limit, years_apart, date_list)

ind_add_oracle (generic function with 1 method)

For each movie $e \in \mathcal{N}$, we let $r_e$ denote the rating of the movie, according to the IMDb.
The ratings take real values between $1$ and $10$.
The *rating budget constraint* is that

$$
\sum_{e \in S} \max (r_e - 5.0, 0) \leq \beta \enspace,
$$
where $\beta$ is a user-defined *rating budget*.
A set $S$ satisfies the rating budget constraint so long as it does not contain too many highly rated movies; indeed, this constraint does not penalize movies which have a rating less than $5.0$.
Observe that the rating budget is a knapsack constraint with coefficients $c_e =  \max (r_e - 5.0, 0)$.
We will set $\beta = 80$.

Below is code that generates the knapsack constraints.

In [34]:
rating_array = Array(movie_info_df[!, :rating])

budget = 130
rating_thresh = 5.0
knapsack_constraints = max.(reshape(rating_array, (1,n)) .- rating_thresh, 0.0) / budget

1×10473 Array{Float64,2}:
 0.0253846  0.0153846  0.0130769  0.00692308  …  0.00923077  0.00769231  0.0

In [35]:
gnd = collect(1:n)

10473-element Array{Int64,1}:
     1
     2
     3
     4
     5
     6
     7
     8
     9
    10
    11
    12
    13
     ⋮
 10462
 10463
 10464
 10465
 10466
 10467
 10468
 10469
 10470
 10471
 10472
 10473

### Running the Algorithms

Now let's run the algorithms.
Since this example is larger, we will set the verbosity to lower levels.

First up, the greedy algorithm.

In [36]:
sol, f_val, oracle_calls, _ = greedy(gnd, f_diff, ind_add_oracle, knapsack_constraints=knapsack_constraints)

(Set([4991, 2613, 1920, 5903, 7012, 6116, 5126, 5602, 3025, 5177  …  5828, 9226, 2084, 2603, 6592, 6606, 331, 5963, 6320, 4637]), 62.96794086462275, 11584, 21476, false)

Next, let's run the Repeated Greedy algorithm.

In [37]:
num_sol = 2
epsilon = 0.1

sol, f_val, oracle_calls, _ = repeated_greedy(gnd, f_diff, ind_add_oracle, knapsack_constraints=knapsack_constraints, num_sol=num_sol, epsilon=epsilon, k=2)

Running repeated greedys
Ground set has 10473 elements
Independence system is 2-system
There are 1 knapsack constraints

Constructing 2 solutions
A grid search for the best density ratio will be run with the parameters:
	beta scaling term is 0.11076923076923079
	error term is 0.1
	bound on largest set is 10473
Obtained solution has value 63.89867855166622
Required 30340 function evaluations and 163261 independence queries


(Set([9568, 9973, 4674, 2785, 3861, 6165, 3728, 5984, 486, 6560  …  4442, 10056, 6531, 4866, 1458, 6663, 7427, 9842, 1915, 1921]), 63.89867855166622, 30340, 163261)

We see that the Repeated Greedy algorithm achieves a better value, but not terribly better.

## The End

This marks the end of the tutorial! 
Hopefully, you understand better how to run the greedy-based optimization algorithms in our `SubmodularGreedy.jl` package.
The key in using this package is creating the marginal gain and marginal independence oracles -- after that, it's mostly plug and play.

If you have questions about this package or comments on possible improvements, please feel free to reach out to me, Christopher Harshaw.
My email is `christopher [dot] harshaw [at] yale.edu` and I would be happy to hear from you.